In [18]:
import torch
import random
import json
import os
import argparse
import os

if os.getcwd() == "/home/user/nikhil/my_refusal_direction/pipeline":
    os.chdir("..")
print(os.getcwd())

from dataset.load_dataset import load_dataset_split, load_dataset

from pipeline.config import Config

/sensei-fs/users/nikhilanand/my_refusal_direction


In [19]:
import transformers
transformers.__version__

'4.42.4'

In [20]:
from pipeline.model_utils.model_factory import construct_model_base
from pipeline.utils.hook_utils import get_activation_addition_input_pre_hook, get_all_direction_ablation_hooks

In [21]:
# from pipeline.submodules.evaluate_jailbreak import evaluate_jailbreak
# from pipeline.submodules.evaluate_loss import evaluate_loss

In [22]:
model_path = "qwen/qwen-1_8b-chat"

In [23]:
model_alias = os.path.basename(model_path)
cfg = Config(model_alias=model_alias, model_path=model_path)

model_base = construct_model_base(cfg.model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from pipeline.run_pipeline import *

In [8]:
harmful_train, harmless_train, harmful_val, harmless_val = load_and_sample_datasets(cfg)

In [9]:
len(harmful_train)

128

In [10]:
harmful_train, harmless_train, harmful_val, harmless_val = filter_data(cfg, model_base, harmful_train, harmless_train, harmful_val, harmless_val)

In [11]:
len(harmful_train)

103

In [12]:
import json
with open('pipeline/runs/qwen-1_8b-chat/direction_metadata.json', 'r') as file:
    data = json.load(file)

pos,layer = data['pos'],data['layer']

In [13]:
direction = torch.load("pipeline/runs/qwen-1_8b-chat/direction.pt")

In [14]:
direction.shape,pos,layer

(torch.Size([2048]), -2, 15)

In [15]:
baseline_fwd_pre_hooks, baseline_fwd_hooks = [], []
ablation_fwd_pre_hooks, ablation_fwd_hooks = get_all_direction_ablation_hooks(model_base, direction)
actadd_fwd_pre_hooks, actadd_fwd_hooks = [(model_base.model_block_modules[layer], get_activation_addition_input_pre_hook(vector=direction, coeff=-1.0))], []

# Ablation will return the pre-hooks to ALL the layers and the output hooks from ALL MLPs and Attns.
# It just returns tuples and we are yet to "register_hook" to each layer.
# Act addn seems to have an empty list for output hooks. 
# It's input hook is only to the optimal layer. Coeff is -1.0 (seems odd, that's basically subtraction)

In [16]:
cfg.artifact_path()

'/sensei-fs/users/nikhilanand/my_refusal_direction/pipeline/my_runs/qwen-1_8b-chat'

In [ ]:
# Completions are basically the LLM's responses to the questions
# This is the moment and I need to save this in my own directory and observe

for dataset_name in cfg.evaluation_datasets:
        generate_and_save_completions_for_dataset(cfg, model_base, baseline_fwd_pre_hooks, baseline_fwd_hooks, 'baseline', dataset_name)
        generate_and_save_completions_for_dataset(cfg, model_base, ablation_fwd_pre_hooks, ablation_fwd_hooks, 'ablation', dataset_name)
        generate_and_save_completions_for_dataset(cfg, model_base, actadd_fwd_pre_hooks, actadd_fwd_hooks, 'actadd', dataset_name)

In [24]:
cfg.evaluation_datasets

('jailbreakbench',)